# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pytz
import argparse
# import jax.numpy as jnp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopy import distance
import datetime
import tilemapbase
from copy import deepcopy
import pickle as pkl
from PIL import Image
import skimage.measure
import math
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
os.environ['PYTHONWARNINGS']='ignore'
import hyperopt
from joblib import Parallel, delayed
import random
random.seed(42)
import scipy
import torch
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from sklearn.model_selection import train_test_split

In [ ]:
source = 'combined'
sensor = 'pm25'
res_time = '1H'
filepath_root = '/scratch/ab9738/pollution_with_sensors/'
# spikes_file = filepath_root+'hotspots/spikes_combined_1H.csv'
# time_high_file = filepath_root+'hotspots/hotspots_combined_temporalhigh_1H.pkl'
# time_low_file = filepath_root+'hotspots/hotspots_combined_temporallow_1H.pkl'
# space_high_file = filepath_root+'hotspots/hotspots_combined_spatialhigh_1H.pkl'
# space_low_file = filepath_root+'hotspots/hotspots_combined_spatiallow_1H.pkl'

# Data Loading

In [ ]:
filepath_data_kai = filepath_root+'data/kaiterra/kaiterra_fieldeggid_{}_current_panel.csv'.format(res_time)
filepath_data_gov = filepath_root+'data/govdata/govdata_{}_current.csv'.format(res_time)
filepath_locs_kai = filepath_root+'data/kaiterra/kaiterra_locations.csv'
filepath_locs_gov = filepath_root+'data/govdata/govdata_locations.csv'

locs_kai = pd.read_csv(filepath_locs_kai, index_col=[0])
locs_kai['Type'] = 'Kaiterra'
locs_gov = pd.read_csv(filepath_locs_gov, index_col=[0])
locs_gov['Type'] = 'Govt'
locs = pd.merge(locs_kai, locs_gov, how='outer',\
                on=['Monitor ID', 'Latitude', 'Longitude', 'Location', 'Type'], copy=False)
data_kai = pd.read_csv(filepath_data_kai, index_col=[0,1], parse_dates=True)[sensor]
data_gov = pd.read_csv(filepath_data_gov, index_col=[0,1], parse_dates=True)[sensor]
data = pd.concat([data_kai, data_gov], axis=0, copy=False)

start_dt = data.index.levels[1][0]
end_dt = data.index.levels[1][-1]

if start_dt.tzname != 'IST':
        if start_dt.tzinfo is None:
            start_dt = start_dt.tz_localize('UTC')
        start_dt = start_dt.tz_convert(pytz.FixedOffset(330))
    
if end_dt.tzname != 'IST':
    if end_dt.tzinfo is None: 
        end_dt = end_dt.tz_localize('UTC')
    end_dt = end_dt.tz_convert(pytz.FixedOffset(330))

# now, filter through the start and end dates
data.sort_index(inplace=True)
data = data.loc[(slice(None), slice(start_dt, end_dt))]

if(source=='govdata'):
    df = data_gov.unstack(level=0)
elif(source=='kaiterra'):
    df = data_kai.unstack(level=0)
else:
    df = data.unstack(level=0)
distances = pd.read_csv('/scratch/ab9738/pollution_with_sensors/data/combined_distances.csv', index_col=[0])
distances = distances.loc[df.columns, df.columns]
distances[distances == 0] = np.nan

In [ ]:
df.replace(0, np.nan, inplace=True)

In [ ]:
df

In [ ]:
locs

Finding random data point to test kriging

In [ ]:
ape = []
i = 0
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
# random_ts = df.sample()
    i+=1
    # if(i==100):
    #     break
    x = locs.loc[df.columns]['Longitude'].values

    y = locs.loc[df.columns]['Latitude'].values

    z = row.values

    x = x[~np.isnan(z)]
    y = y[~np.isnan(z)]
    z = z[~np.isnan(z)]
    
    if(len(x)<30):
        continue

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(
        x, y, z, test_size=0.2, random_state=42
    )

    # Do the kriging experiment

    # OK = OrdinaryKriging(
    #     x_train,
    #     y_train,
    #     z_train,
    #     variogram_model="spherical",
    #     verbose=False,
    #     enable_plotting=False,
    # )
    UK = UniversalKriging(
    x_train,
    y_train,
    z_train,
    variogram_model="linear",
    drift_terms=["regional_linear"],
    )

    z_pred, ss_pred = UK.execute("points", x_test, y_test)
    ape.append(np.abs((z_test-z_pred)/z_test))
ape_arr = np.concatenate(ape)
mape = np.mean(ape_arr)
mape

In [ ]:
# gridx = np.arange(76.85, 77.65, 0.01)
# gridy = np.arange(28.2, 29.0, 0.01)

# z_grid, ss_grid = OK_train.execute("grid", gridx, gridy[::-1])

# plt.imshow(z_grid)
# plt.show()

In [ ]:
len(df)